In [40]:
%pip install langgraph langchain langchain_openai langchain_community
%pip install -q langfuse

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [41]:
import os, getpass
from langfuse import Langfuse
from langfuse.callback import CallbackHandler

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("LANGFUSE_PUBLIC_KEY")
_set_env("LANGFUSE_SECRET_KEY")
_set_env("LANGFUSE_HOST")

langfuse = Langfuse(
  secret_key=os.environ["LANGFUSE_SECRET_KEY"],
  public_key=os.environ["LANGFUSE_PUBLIC_KEY"],
  host=os.environ["LANGFUSE_HOST"]
)

langfuse_handler = CallbackHandler()

In [42]:
from typing import TypedDict, List, Optional, Dict, Any
from langgraph.graph import MessagesState
import json

class Draft(TypedDict):
    title: str
    body: str
    images: List[str]

class State(MessagesState):
    prompt: str
    topic: Optional[str]
    sources: Optional[List[Dict[str, Any]]]
    reasoning: Optional[List[str]]
    draft: Optional[Draft]

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    streaming=True,
    verbose=True,
    callbacks=[langfuse_handler],
)

tavily = TavilySearchResults(max_results=5)


In [44]:
def get_user_prompt_node(state: State) -> State:
    '''
    Get the user prompt node.
    '''
    prompt = input(
        "Inserisci il prompt per l'articolo: "
    )
    state["prompt"] = prompt
    return state

In [45]:
def router_node(state: State) -> State:
    '''
    Route the user prompt to the appropriate node.
    '''
    if not state["prompt"]:
        raise ValueError("Prompt is empty.")
    
    system_message = SystemMessage("Estrai dal seguente messaggio il topic cui fa riferimento. Rispondi solo con il topic estratto.")
    human_message = HumanMessage(content=state["prompt"])

    response = llm([system_message, human_message])

    topic = response.content.strip()
    state["topic"] = topic
    return state

In [46]:
def search_sources_node(state: State) -> State:
    '''
    Search for sources using Tavily.
    '''
    if not state["topic"]:
        raise ValueError("Topic is required to search for sources.")
    
    results = tavily.invoke(state["topic"])

    # Format
    formatted_search_docs = "\n\n---\n\n".join(
        [
            f'<Document href="{doc["title"]}\n{doc["url"]}">\n{doc["content"]}\n{doc["score"]}</Document>'
            for doc in results if doc["score"] > 0.6
        ]
    )
    print(f"Formatted search results:\n{formatted_search_docs}")
    
    state["sources"] = formatted_search_docs
    return state
    

In [52]:
def verify_sources_node(state: State) -> State:
    '''
    Verify the sources with the user.
    '''
    if not state["sources"]:
        raise ValueError("Sources are required for verification.")
    
    system_message = SystemMessage(f'''
        Sei un esperto di verifica delle fonti. Ti fornirò una lista di risultati di ricerca. La tua missione è quella di
        verificare se le fonti sono attendibili e verosimili. Per ogni fonte, se la fonte è attendibile e verosimile,
        aggiungi l'url alla lista delle fonti verificate. Se la fonte non è attendibile o verosimile, spiega perché e aggiungi
        il motivo per cui quell'articolo non è attendibile alla lista "reasoning" (includi l'url).
        Alla fine, restituisci due liste: una con le fonti verificate e una con i motivi per cui le fonti non sono attendibili.
        Non restituire altro. Solo "sources" e "reasoning".
        Il topic è: {state["topic"]}
        Queste sono le fonti: {state["sources"]}                
    ''')
    response = llm([system_message])
    print(f"Response:\n{response.content}")

    response_data = json.loads(response.content)
    sources = json.loads(response_data.get("sources", "{}")).get("sources", [])
    reasoning = json.loads(response_data.get("reasoning", "{}")).get("reasoning", [])

    print(f"Sources:\n{sources}")
    print(f"Reasoning:\n{reasoning}")

    state["sources"] = response.content.strip()

    return state

In [48]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)
builder.add_node("get_user_prompt", get_user_prompt_node)
builder.add_node("router", router_node)
builder.add_node("search_sources", search_sources_node)
builder.add_node("verify_sources", verify_sources_node)

builder.add_edge(START, "get_user_prompt")
builder.add_edge("get_user_prompt", "router")
builder.add_edge("router", "search_sources")
builder.add_edge("search_sources", "verify_sources")
builder.add_edge("verify_sources", END)

In [53]:
from IPython.display import Image, display

graph = builder.compile().with_config(
    callbacks=[langfuse_handler],
)
# display(Image(graph.get_graph().draw_mermaid_png()))

init_state: State = {"prompt": ""}
graph.invoke(init_state)

Formatted search results:
<Document href="Here's how to increase your chances of nabbing a Switch 2 preorder
https://www.theverge.com/tech/653010/nintendo-switch-2-preorder-tips-tricks-release-date-price-availability-how-to-buy">
At launch, the Nintendo Switch 2 will be available in two distinct configurations: a $449.99 standard version and a $499.99 bundle that includes the latest Mario Kart title, Mario Kart World. Nintendo is on record saying the bundle will only be available through fall 2025 (or while supplies last), which is a shame, considering picking up the console / game combo saves you $30 on what is likely to be the Switch 2’s biggest launch title. If you don’t manage to secure the bundle, you can pick up [...] Following a brief delay stemming from the Trump’s administration’s ongoing tariff policy, Nintendo is finally opening up preorders for the Switch 2. Most major retailers — including Walmart, Best Buy, and Target — will open preorders at 12AM ET on April 24th / 9PM P

JSONDecodeError: Expecting value: line 1 column 1 (char 0)